
# 对大数据进行预处理

以占领华尔街推特数据为例



![image.png](./images/author.png)

## 字节（Byte /bait/）

计算机信息技术用于计量存储容量的一种计量单位，通常情况下一字节等于有八位， [1]  也表示一些计算机编程语言中的数据类型和语言字符。
- 1B（byte，字节）= 8 bit；
- 1KB=1000B；1MB=1000KB=1000×1000B。其中1000=10^3。
- 1KB（kilobyte，千字节）=1000B= 10^3 B；
- 1MB（Megabyte，兆字节，百万字节，简称“兆”）=1000KB= 10^6 B；
- 1GB（Gigabyte，吉字节，十亿字节，又称“千兆”）=1000MB= 10^9 B；

## 按照Chunk读取数据并进行处理

Lazy Method for Reading Big File in Python?

In [1]:
# 按行读取数据
line_num = 0
cops_num = 0
with open('/Users/datalab/bigdata/cjc/ows-raw.txt', 'r') as f:
    for i in f:
        line_num += 1
        if 'cops' in i:
            cops_num += 1
        if line_num % 100000 ==0:
            print(line_num)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

In [2]:
line_num

6911408

In [4]:
cops_num/line_num

0.011413448605551865

In [1]:
bigfile = open('/Users/datalab/bigdata/cjc/ows-raw.txt', 'r')
chunkSize = 1000000
chunk = bigfile.readlines(chunkSize)
print(len(chunk))
# with open("../data/ows_tweets_sample.txt", 'w') as f:
#     for i in chunk:
#         f.write(i)  

2754


In [5]:
bigfile.readlines?

In [8]:
5%5

0

In [5]:
# https://stackoverflow.com/questions/519633/lazy-method-for-reading-big-file-in-python?lq=1
import csv
bigfile = open('/Users/datalab/bigdata/cjc/ows-raw.txt', 'r')
chunkSize = 10**8
chunk = bigfile.readlines(chunkSize)
num_chunk, num_lines = 0, 0
while chunk:
    lines = csv.reader((line.replace('\x00','') for line in chunk), 
                       delimiter=',', quotechar='"')
    #do sth.
    num_lines += len(list(lines))
    if num_chunk % 5 ==0:
        print(num_chunk, num_lines)
    num_chunk += 1
    chunk = bigfile.readlines(chunkSize) # read another chunk

0 262665
5 1574666
10 2880857
15 4189419
20 5492578
25 6602141


In [4]:
num_lines

6602141

## 用Pandas的get_chunk功能来处理亿级数据

> 只有在超过5TB数据量的规模下，Hadoop才是一个合理的技术选择。

In [14]:
import pandas as pd

f = open('/Users/datalab/bigdata/cjc/ows-raw.txt',encoding='utf-8')
reader = pd.read_table(f,  sep=',',  quotechar='"', iterator=True, error_bad_lines=False) #跳过报错行
chunkSize = 100000
chunk = reader.get_chunk(chunkSize)
len(chunk)

#pd.read_table?

100000

In [15]:
chunk.head()

,Twitter ID,Text,Profile Image URL,Day,Hour,Minute,Created At,Geo,From User,From User ID,Language,To User,To User ID,Source
0,121813144174727168,RT @AnonKitsu: ALERT!!!!!!!!!!COPS ARE KETTLIN...,http://a2.twimg.com/profile_images/1539375713/...,2011-10-06,5,4,2011-10-06 05:04:51,N;,Anonops_Cop,401240477,en,NaN,0,&lt;a href=&quot;http://twitter.com/&quot;&gt;...
1,121813146137657344,@jamiekilstein @allisonkilkenny Interesting in...,http://a2.twimg.com/profile_images/1574715503/...,2011-10-06,5,4,2011-10-06 05:04:51,N;,KittyHybrid,34532053,en,jamiekilstein,2149053,&lt;a href=&quot;http://twitter.com/&quot;&gt;...
2,121813150000619521,@Seductivpancake Right! Those guys have a vict...,http://a1.twimg.com/profile_images/1241412831/...,2011-10-06,5,4,2011-10-06 05:04:52,N;,nerdsherpa,95067344,en,Seductivpancake,19695580,&lt;a href=&quot;http://www.echofon.com/&quot;...
3,121813150701072385,RT @bembel &quot;Occupy Wall Street&quot; als ...,http://a0.twimg.com/profile_images/1106399092/...,2011-10-06,5,4,2011-10-06 05:04:52,N;,hamudistan,35862923,en,NaN,0,&lt;a href=&quot;http://levelupstudio.com&quot...
4,121813163778899968,#ows White shirt= Brown shirt.,http://a2.twimg.com/profile_images/1568117871/...,2011-10-06,5,4,2011-10-06 05:04:56,N;,kl_knox,419580636,en,NaN,0,&lt;a href=&quot;http://twitter.com/&quot;&gt;...


In [2]:
import pandas as pd

f = open('/Users/datalab/bigdata/cjc/ows-raw.txt',encoding='utf-8')
reader = pd.read_table(f,  sep=',',  quotechar='"', 
                       iterator=True, error_bad_lines=False) #跳过报错行
chunkSize = 100000
loop = True
#data = []
num_chunk, num_lines = 0, 0
while loop:
    try:
        chunk = reader.get_chunk(chunkSize)
        # dat = data_cleaning_funtion(chunk) # do sth.
        num_lines += len(chunk)
        print(num_chunk, num_lines)
        num_chunk +=1
        #data.append(dat) 
    except StopIteration:
        loop = False
        print("Iteration is stopped.")
#df = pd.concat(data, ignore_index=True)

0 100000
1 200000
2 300000
3 400000
4 500000
5 600000
6 700000
7 800000
8 900000
9 1000000
10 1100000
11 1200000
12 1300000
13 1400000
14 1500000
15 1600000
16 1700000
17 1800000
18 1900000
19 2000000
20 2100000
21 2200000
22 2300000
23 2400000
24 2500000
25 2600000
26 2700000
27 2800000
28 2900000
29 3000000
30 3100000
31 3200000
32 3300000
33 3400000
34 3500000
35 3600000
36 3700000
37 3800000
38 3900000
39 4000000
40 4100000
41 4200000
42 4300000
43 4400000
44 4500000
45 4600000
46 4700000
47 4800000
48 4900000
49 5000000


b'Skipping line 5051743: expected 14 fields, saw 15\n'


50 5100000
51 5200000


b'Skipping line 5254718: expected 14 fields, saw 15\n'
b'Skipping line 5281095: expected 14 fields, saw 15\n'


52 5300000
53 5400000


...

59 6000000
60 6100000
61 6200000


b'Skipping line 6229621: expected 14 fields, saw 16\nSkipping line 6245861: expected 14 fields, saw 17\n'
b'Skipping line 6278728: expected 14 fields, saw 15\n'


62 6300000


b'Skipping line 6350262: expected 14 fields, saw 15\n'
b'Skipping line 6387321: expected 14 fields, saw 15\nSkipping line 6388879: expected 14 fields, saw 15\n'


63 6400000
64 6500000
65 6600000
66 6602120
Iteration is stopped.


![image.png](./images/end.png)